<a href="https://colab.research.google.com/github/pollyjuice74/REU-LDPC-Project/blob/main/ECCT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/pollyjuice74/ECCT
!git clone https://github.com/pollyjuice74/REU-LDPC-Project
!git clone https://github.com/pollyjuice74/gnn-decoder

!pip install sionna
#!pip install torch torch-geometric

Cloning into 'ECCT'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 101 (delta 53), reused 31 (delta 9), pack-reused 0
Receiving objects: 100% (101/101), 84.62 KiB | 893.00 KiB/s, done.
Resolving deltas: 100% (53/53), done.
Cloning into 'REU-LDPC-Project'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 96 (delta 33), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (96/96), 48.14 KiB | 849.00 KiB/s, done.
Resolving deltas: 100% (33/33), done.
Cloning into 'gnn-decoder'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 52 (delta 26), reused 35 (delta 13), pack-reused 0
Receiving objects: 100% (52/52), 859.06 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (26/26), done.
    

In [3]:
import os
# Rename folders
if os.path.exists('REU-LDPC-Project'):
  os.rename('REU-LDPC-Project', 'REU_LDPC_Project')
if os.path.exists('gnn-decoder'):
  os.rename('gnn-decoder', 'gnn_decoder')

In [ ]:
for i in enc.pcm:
  print(i)

In [ ]:
from sionna.fec.ldpc.encoding import LDPC5GEncoder
from sionna.fec.ldpc.decoding import LDPC5GDecoder

from REU_LDPC_Project.channel import E2EModel

# from ECCT.Model import ECC_Transformer
from ECCT.Codes import FER, BER # x_pred, x_gt
from ECCT.args import pass_args

from gnn_decoder.gnn import LDPC5GGNN

n, k = (100, 90)
args = pass_args()

enc = LDPC5GEncoder(k,n)
dec = LDPC5GDecoder(enc)

# Models
gnn = LDPC5GGNN(enc)
ecct = ECC_Transformer(args, enc, dec)

model = None
ch = E2EModel(enc, dec, model)



In [ ]:
class ECC_Transformer(nn.Module):
    def __init__(self, args, encoder, decoder, dropout=0):
        super(ECC_Transformer, self).__init__()
        ####
        code = args.code
        c = copy.deepcopy
        attn = MultiHeadedAttention(args.h, args.d_model)
        ff = PositionwiseFeedForward(args.d_model, args.d_model*4, dropout)

        # 5G Compliant encoder/decoder
        self.encoder5G = encoder
        self.decoder5G = decoder

        self.src_embed = torch.nn.Parameter(torch.empty(self.encoder5G._n + self.encoder5G.pcm.shape[0], args.d_model)) #(code.n + code.pc_matrix.size(0), args.d_model)))
        self.decoder = Encoder(EncoderLayer(
            args.d_model, c(attn), c(ff), dropout), args.N_dec)
        self.oned_final_embed = torch.nn.Sequential(
            *[nn.Linear(args.d_model, 1)])
        self.out_fc = nn.Linear(self.encoder5G._n + self.encoder5G.pcm.shape[0], self.encoder5G._n)

        self.get_mask(code)
        logging.info(f'Mask:\n {self.src_mask}')
        ###
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, magnitude, syndrome):
        emb = torch.cat([magnitude, syndrome], -1).unsqueeze(-1)
        emb = self.src_embed.unsqueeze(0) * emb
        emb = self.decoder(emb, self.src_mask)
        return self.out_fc(self.oned_final_embed(emb).squeeze(-1))

    def loss(self, z_pred, z2, y):
        loss = F.binary_cross_entropy_with_logits(
            z_pred, sign_to_bin(torch.sign(z2)))
        x_pred = sign_to_bin(torch.sign(-z_pred * torch.sign(y)))
        return loss, x_pred

    def get_mask(self, code, no_mask=False):
        if no_mask:
            self.src_mask = None
            return

        def build_mask(code, encoder):
            mask_size = self.encoder5G._n +self.encoder5G.pcm.shape[0]
            mask = torch.eye(mask_size, mask_size)
            for ii in range(self.encoder5G.pcm.shape[0]):
                idx = torch.where(self.encoder5G.pcm[ii] > 0)[0] #code.pc_matrix[ii]
                for jj in idx:
                    for kk in idx:
                        if jj != kk:
                            mask[jj, kk] += 1
                            mask[kk, jj] += 1
                            mask[self.encoder5G._n + ii, jj] += 1
                            mask[jj, self.encoder5G._n + ii] += 1
            src_mask = ~ (mask > 0).unsqueeze(0).unsqueeze(0)
            return src_mask
        src_mask = build_mask(code, self.encoder5G)
        mask_size = self.encoder5G._n +self.encoder5G.pcm.shape[0]
        a = mask_size ** 2
        logging.info(
            f'Self-Attention Sparsity Ratio={100 * torch.sum((src_mask).int()) / a:0.2f}%, Self-Attention Complexity Ratio={100 * torch.sum((~src_mask).int())//2 / a:0.2f}%')
        self.register_buffer('src_mask', src_mask)


